In [60]:
import pandas as pd

# 1. Generar un DataFrame con los datos de los tres ficheros
ficheros = ['emisiones-2016.csv', 'emisiones-2018.csv', 'emisiones-2019.csv']
df_list = []

for fichero in ficheros:
    df = pd.read_csv(fichero, delimiter=';')  
    df_list.append(df)
df_completo = pd.concat(df_list, ignore_index=True)

print(df_completo)

      PROVINCIA  MUNICIPIO  ESTACION  MAGNITUD PUNTO_MUESTREO   ANO  MES  \
0            28         79         4         1  28079004_1_38  2016    1   
1            28         79         4         1  28079004_1_38  2016    2   
2            28         79         4         1  28079004_1_38  2016    3   
3            28         79         4         1  28079004_1_38  2016    4   
4            28         79         4         1  28079004_1_38  2016    5   
...         ...        ...       ...       ...            ...   ...  ...   
5449         28         79        60        14  28079060_14_6  2019    8   
5450         28         79        60        14  28079060_14_6  2019    9   
5451         28         79        60        14  28079060_14_6  2019   10   
5452         28         79        60        14  28079060_14_6  2019   11   
5453         28         79        60        14  28079060_14_6  2019   12   

       D01 V01    D02  ...   D27  V27   D28  V28   D29  V29    D30  V30   D31  \
0     

In [38]:
import pandas as pd
# 2. Filtrar las columnas del DataFrame
columnas_interes = ['ESTACION', 'MAGNITUD', 'ANO', 'MES'] + [f'D{i:02d}' for i in range(1, 32)] 
df_completo = df_completo[columnas_interes]

print(df_completo)


      ESTACION  MAGNITUD   ANO  MES   D01    D02    D03   D04   D05   D06  \
0            4         1  2016    1   8.0    7.0    6.0   6.0   7.0   6.0   
1            4         1  2016    2  12.0   13.0    9.0   9.0  11.0   9.0   
2            4         1  2016    3  11.0   10.0    9.0   9.0   7.0   8.0   
3            4         1  2016    4   8.0    9.0    9.0   8.0   8.0   9.0   
4            4         1  2016    5   7.0    8.0    9.0   9.0   8.0   8.0   
...        ...       ...   ...  ...   ...    ...    ...   ...   ...   ...   
5449        60        14  2019    8  94.0  104.0  106.0  99.0  77.0  82.0   
5450        60        14  2019    9  88.0   82.0   80.0  92.0  79.0  75.0   
5451        60        14  2019   10  44.0   75.0   44.0  54.0  65.0  68.0   
5452        60        14  2019   11  41.0   55.0   79.0  65.0  64.0  51.0   
5453        60        14  2019   12  47.0   53.0   45.0  44.0  49.0  41.0   

      ...   D22   D23    D24   D25   D26   D27   D28   D29    D30   D31  
0

In [62]:
import pandas as pd
# 3. Reestructurar el DataFrame para que los valores de los contaminantes 
# de las columnas de los días aparezcan en una única columna
df_reshaped = df_completo.melt(id_vars=['ESTACION', 'MAGNITUD', 'ANO', 'MES'], 
                               value_vars=[f'D{i:02d}' for i in range(1, 32)], 
                               var_name='DIA', 
                               value_name='VALOR')

print(df_reshaped)


        ESTACION  MAGNITUD   ANO  MES  DIA  VALOR
0              4         1  2016    1  D01    8.0
1              4         1  2016    2  D01   12.0
2              4         1  2016    3  D01   11.0
3              4         1  2016    4  D01    8.0
4              4         1  2016    5  D01    7.0
...          ...       ...   ...  ...  ...    ...
169069        60        14  2019    8  D31   98.0
169070        60        14  2019    9  D31    0.0
169071        60        14  2019   10  D31   47.0
169072        60        14  2019   11  D31    0.0
169073        60        14  2019   12  D31    4.0

[169074 rows x 6 columns]


In [122]:
import pandas as pd

# 1. Generar un DataFrame con los datos de los tres ficheros
ficheros = ['emisiones-2016.csv', 'emisiones-2018.csv', 'emisiones-2019.csv']
df_list = []

for fichero in ficheros:
    df = pd.read_csv(fichero, delimiter=';')  # Asegúrate de usar el delimitador correcto
    df_list.append(df)
df_completo = pd.concat(df_list, ignore_index=True)

# 2. Filtrar las columnas del DataFrame
columnas_interes = ['ESTACION', 'MAGNITUD', 'ANO', 'MES'] + [f'D{i:02d}' for i in range(1, 32)]  # D01 a D31
df_completo = df_completo[columnas_interes]

# 3. Reestructurar el DataFrame para que los valores de los contaminantes de las columnas de los días aparezcan en una única columna
df_reshaped = df_completo.melt(id_vars=['ESTACION', 'MAGNITUD', 'ANO', 'MES'], 
                               value_vars=[f'D{i:02d}' for i in range(1, 32)], 
                               var_name='DIA', 
                               value_name='VALOR')

# 4. Extraer el número del día y convertirlo en un entero
df_reshaped['DIA'] = df_reshaped['DIA'].str.extract(r'(\d+)').astype(int)

# Filtrar filas con valores nulos en las columnas relevantes
df_reshaped_clean = df_reshaped.dropna(subset=['ANO', 'MES', 'DIA'])

# Limpiar los días fuera del rango (1-31)
df_reshaped_clean = df_reshaped_clean[(df_reshaped_clean['DIA'] >= 1) & (df_reshaped_clean['DIA'] <= 31)]

df_reshaped_clean['FECHA_STR'] = df_reshaped_clean['ANO'].astype(str) + '-' + df_reshaped_clean['MES'].apply(lambda x: f'{x:02d}') + '-' + df_reshaped_clean['DIA'].apply(lambda x: f'{x:02d}')

df_reshaped_clean['FECHA'] = pd.to_datetime(df_reshaped_clean['FECHA_STR'], errors='coerce')

print(df_reshaped_clean)

        ESTACION  MAGNITUD   ANO  MES  DIA  VALOR   FECHA_STR      FECHA
0              4         1  2016    1    1    8.0  2016-01-01 2016-01-01
1              4         1  2016    2    1   12.0  2016-02-01 2016-02-01
2              4         1  2016    3    1   11.0  2016-03-01 2016-03-01
3              4         1  2016    4    1    8.0  2016-04-01 2016-04-01
4              4         1  2016    5    1    7.0  2016-05-01 2016-05-01
...          ...       ...   ...  ...  ...    ...         ...        ...
169069        60        14  2019    8   31   98.0  2019-08-31 2019-08-31
169070        60        14  2019    9   31    0.0  2019-09-31        NaT
169071        60        14  2019   10   31   47.0  2019-10-31 2019-10-31
169072        60        14  2019   11   31    0.0  2019-11-31        NaT
169073        60        14  2019   12   31    4.0  2019-12-31 2019-12-31

[169074 rows x 8 columns]


In [131]:
import numpy as np

# 5. Eliminar las filas con fechas no válidas usando 
df_final = df_reshaped_clean[~np.isnat(df_reshaped_clean['FECHA'].values)]


df_final = df_final.sort_values(['ESTACION', 'MAGNITUD', 'FECHA'])

df_final = df_final.reset_index(drop=True)

print(df_final)

        ESTACION  MAGNITUD   ANO  MES  DIA  VALOR   FECHA_STR      FECHA
0              4         1  2016    1    1    8.0  2016-01-01 2016-01-01
1              4         1  2016    1    2    7.0  2016-01-02 2016-01-02
2              4         1  2016    1    3    6.0  2016-01-03 2016-01-03
3              4         1  2016    1    4    6.0  2016-01-04 2016-01-04
4              4         1  2016    1    5    7.0  2016-01-05 2016-01-05
...          ...       ...   ...  ...  ...    ...         ...        ...
166038        60        14  2019   12   27   17.0  2019-12-27 2019-12-27
166039        60        14  2019   12   28   13.0  2019-12-28 2019-12-28
166040        60        14  2019   12   29   14.0  2019-12-29 2019-12-29
166041        60        14  2019   12   30    5.0  2019-12-30 2019-12-30
166042        60        14  2019   12   31    4.0  2019-12-31 2019-12-31

[166043 rows x 8 columns]
